In [1]:
from distributed_processing.serializers import DummySerializer
from distributed_processing.worker import Worker
from distributed_processing.filesystem_connector import FileSystemConnector

In [2]:
CURRO = True

if CURRO:
    NS_PATH ="G:\\fs_namespaces\\prueba_distribuida"
    #NS_PATH ="C:\\fs_namespaces\\prueba_distribuida"

else:
    NS_PATH = "/home/augusto/python/notebooks/fs_namespaces/prueba_distribuida"


In [3]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [4]:
fs_connector = FileSystemConnector(NS_PATH)
fs_connector.with_watchdog = True
# Podemos limpiar el namespace por si quedan restos de ejecuciones anteriores.
#fs_connector.clean_namespace()

server = Worker(DummySerializer(), fs_connector)


Worker id: fs_client_16


In [5]:
def add(x, y):
    return x + y

def mul(x, y):
    return x * y

def div(x, y):
    return x / y

func_dict1 = {"add" : add,
              "mul" : mul,
              "div" : div}

server.add_requests_queue("cola_1", func_dict1)


In [6]:
def hola(nombre, calificativo="listo"):
    return f"Hola {nombre}, eres muy {calificativo}"

func_dict2 = {"hola": hola}

# Los métodos disponibles en cola_2 no los vamos a hacer públicos
server.add_requests_queue("cola_2", func_dict2, register=False)

In [7]:
server.add_python_eval()

"""
# Equivalente a:
import dill
import base64

def eval_py_function(str_fn, args=[], kwargs={}):
    "str_fn encoded en base64 ascii"
    return dill.loads(base64.b64decode(str_fn))(*args, **kwargs)

server.add_requests_queue("py_eval", {"eval_py_function": eval_py_function})
"""

_

''

In [8]:
server.queues_to_register

{'requests_cola_1', 'requests_py_eval'}

In [9]:
# Publicamos todos los métodos de las colas 
server.update_methods_registry()

Method add published as available for queues: requests_cola_1
Method mul published as available for queues: requests_cola_1
Method div published as available for queues: requests_cola_1
Method eval_py_function published as available for queues: requests_py_eval


In [ ]:
server.run()